In [39]:
import sys
import numpy as np
import pandas as pd
import time
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from smt.sampling_methods import LHS
tqdm.pandas()
from collections import defaultdict
sys.version

'3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 15:50:08) [MSC v.1916 64 bit (AMD64)]'

# Create Scope input

In [40]:
"""
PROSPECT
Cab
Cca
Cdm
Cw
Cs
Cant
Cp
Cbc
N
rho_thermal
tau_thermal

Leaf_Biochemical
Vcmax25
BallBerrySlope
BallBerry0
Type
kV
Rdparam
Kn0
Knalpha
Knbeta

Leaf_Biochemical_magnani
Tyear
beta
kNPQs
qLs
stressfactor

Fluorescence
fqe

Soil
spectrum
rss
rs_thermal
cs
rhos
lambdas
SMC
BSMBrightness
BSMlat
BSMlon

Canopy
LAI
hc
LIDFa
LIDFb
leafwidth
Cv
crowndiameter

Meteo
z
Rin
Ta
Rli
p
ea
u
Ca
Oa

Aerodynamic
zo
d
Cd
rb
CR
CD1
Psicor
CSSOIL
rbs
rwc

timeseries
startDOY
endDOY
LAT
LON
timezn

Angles
tts
tto
psi
"""

param_ranges = \
    {
        'Cab':(11, 90),
        'Cca':(0, 40),
        'Cdm':(0, 0.05),
        'Cw':(0.0, 0.1),
        'Cs':(0, 0.9),
        'Cant':(0, 40),
        'Cp':(0,0),
        'Cbc':(0,0),
        'N':(1, 2.5),
        'rho_thermal':(0.01, 0.01),
        'tau_thermal':(0.01, 0.01),
        'Vcmax25':(100, 100),
        'BallBerrySlope':(8, 8), #
        'BallBerry0':(0.01, 0.01),
        'Type':(0, 0),
        'kV':(0.64, 0.64),
        'Rdparam':(0.015, 0.015),
        'Kn0':(2.48, 2.48),
        'Knalpha':(2.83, 2.83),
        'Knbeta':(0.144, 0.144),
        'Tyear':(15, 15),
        'beta':(0.507, 0.507),
        'kNPQs':(0, 0),
        'qLs':(1, 1),
        'stressfactor':(1, 1),
        'fqe':(0.01, 0.01),
        'spectrum':(1, 1),
        'rss':(500, 500),
        'rs_thermal':(0.06, 0.06),
        'cs':(1.18E+03, 1.18E+03),
        'rhos':(1.18E+03, 1.18E+03),
        'lambdas':(1.55, 1.55),
        'SMC':(0.01, 0.7),
        'BSMBrightness':(0.01, 0.9),
        'BSMlat':(20, 40),
        'BSMlon':(45, 65),
        'LAI':(0, 9),
        'hc':(0.1, 2),
        'LIDFa':(-1, 1),
        'LIDFb':(-1, 1),
        'leafwidth':(0.01, 0.01),
        'Cv':(1,1),
        'crowndiameter':(1,1),
        'z':(5,5),
        'Rin':(0,1400),
        'Ta':(-10,50),
        'Rli':(0,400),
        'p':(500, 1030),
        'ea':(0,125),
        'u':(0,25),
        'Ca':(410, 410),
        'Oa':(209, 209),
        'zo':(0.246, 0.246),
        'd':(1.34, 1.34),
        'Cd':(0.3, 0.3),
        'rb':(10, 10),
        'CR':(0.35, 0.35),
        'CD1':(20.6, 20.6),
        'Psicor':(0.2, 0.2),
        'CSSOIL':(0.01, 0.01),
        'rbs':(10, 10),
        'rwc':(0, 0),
        'startDOY':(0,0), #
        'endDOY':(0,0 ), #
        'LAT':(0,0 ), #
        'LON':(0,0 ), #
        'timezn':(0,0 ), #
        'tts':(0, 85),
        'tto':(0, 0),
        'psi':(90, 90)   
    }



In [45]:
def latin_hypercube_sampling(parameter_ranges, n_samples=10):
    
    param_ranges = np.array([parameter_ranges[x] for x in parameter_ranges.keys()], dtype="object")
    param_names = list(parameter_ranges.keys())
    sampling = LHS(xlimits=param_ranges)

    samples = sampling(n_samples)
    param_configurations = [dict(zip(param_names,sample)) for sample in samples]
    
    return param_configurations, constant_params

def lhs_to_scope_input(param_configurations):
    param_names = param_configurations[0].keys()
#     output = {param_name:[] for param_name in param_names}
    output = defaultdict(list)
    for d in param_configurations:
        for k, v in d.items():
            output[k].append(v)
            
    output = dict(output)
    return output

In [46]:
params = latin_hypercube_sampling(param_ranges)
lhs_to_scope_input(params)

{'Cab': [54.45,
  70.25,
  30.75,
  86.05,
  78.15,
  38.650000000000006,
  14.95,
  22.85,
  62.35000000000001,
  46.550000000000004],
 'Cca': [18.0,
  34.0,
  22.0,
  6.000000000000001,
  2.0,
  26.000000000000007,
  30.0,
  14.000000000000002,
  10.0,
  38.0],
 'Cdm': [0.022500000000000003,
  0.0475,
  0.0075000000000000015,
  0.027500000000000004,
  0.037500000000000006,
  0.0175,
  0.03250000000000001,
  0.0125,
  0.0025000000000000005,
  0.04250000000000001],
 'Cw': [0.05500000000000001,
  0.025,
  0.095,
  0.035,
  0.005000000000000001,
  0.08500000000000002,
  0.045000000000000005,
  0.06500000000000002,
  0.07500000000000001,
  0.015000000000000003],
 'Cs': [0.855,
  0.405,
  0.7650000000000001,
  0.675,
  0.225,
  0.045000000000000005,
  0.5850000000000002,
  0.13500000000000004,
  0.49500000000000005,
  0.31500000000000006],
 'Cant': [14.000000000000002,
  38.0,
  30.0,
  26.000000000000007,
  6.000000000000001,
  2.0,
  34.0,
  22.0,
  10.0,
  18.0],
 'Cp': [0.0, 0.0, 0.0, 

In [16]:
params

['Cab',
 'Cca',
 'Cdm',
 'Cw',
 'Cs',
 'Cant',
 'Cp',
 'Cbc',
 'N',
 'rho_thermal',
 'tau_thermal',
 'Vcmax25',
 'BallBerrySlope',
 'BallBerry0',
 'Type',
 'kV',
 'Rdparam',
 'Kn0',
 'Knalpha',
 'Knbeta',
 'Tyear',
 'beta',
 'kNPQs',
 'qLs',
 'stressfactor',
 'fqe',
 'spectrum',
 'rss',
 'rs_thermal',
 'cs',
 'rhos',
 'lambdas',
 'SMC',
 'BSMBrightness',
 'BSMlat',
 'BSMlon',
 'LAI',
 'hc',
 'LIDFa',
 'LIDFb',
 'leafwidth',
 'Cv',
 'crowndiameter',
 'z',
 'Rin',
 'Ta',
 'Rli',
 'p',
 'ea',
 'u',
 'Ca',
 'Oa',
 'zo',
 'd',
 'Cd',
 'rb',
 'CR',
 'CD1',
 'Psicor',
 'CSSOIL',
 'rbs',
 'rwc',
 'startDOY',
 'endDOY',
 'LAT',
 'LON',
 'timezn',
 'tts',
 'tto',
 'psi']

# Download RS data